<a href="https://colab.research.google.com/github/MiladQolami/BasicNeuralDataAnalysis/blob/main/MainCode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Basics of Spike Train visualization

## Loadng data and defining initial parameters


In [4]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.io as sio

spikes = sio.loadmat('SpikesIT.mat')
spikes

{'__header__': b'MATLAB 5.0 MAT-file, Platform: PCWIN64, Created on: Mon Nov 14 20:52:04 2022',
 '__version__': '1.0',
 '__globals__': [],
 'Spikes': array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=uint8),
 'TrialTags': array([[ 84,   2,  92, 101,  90,  38, 137, 114, 106,  98,  63, 147,  13,
          57, 139,  26, 144,  31, 106, 109, 102, 143, 135, 113,  72,  38,
          24, 127, 144, 112, 113,  10, 104, 119,  77,  48, 123, 144,  35,
          45,  85, 130,  98,  69, 151, 106,  94,  18,  88,   1,  15,  49,
          27,  78, 120,  95,  30,  84, 114,  61,  96, 145,  51,  89,  32,
           9, 116,  71,   6, 109, 108,  68, 101,  89,  44,  83,  20,  15,
         104, 142, 105,  59,   6, 135,  97,  76,  94,  36,  41, 139,  19,
          35,  56,  25,  40,  15,  26, 129, 121,  49, 135,  64, 138,  52,
          83, 108,  